# Мультиклассовая классификация

В данном ноутбуке рассмотрим подход к задаче с точки зрения мультиклассовой классификации. То есть, мы не будем учитывать порядок классов, а попробуем просто спрогнозировать их, как равнозначные.

В ноутбуке с анализом данных, исходя из корреляций признаков объектов и целевого значения, удалось установить, что многие признаки имеют высокую корреляцию с таргетом. Таким образом, стоит попробовать модель LogisticRegression.

Также рассмотрим ансабль на деревьях - случайный лес и градиентный бустинг.

In [1]:
import pandas as pd
import numpy as np

In [3]:
df_train = pd.read_csv("/content/drive/MyDrive/intern_task_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/intern_task_test.csv")

Предварительно создадим функцию для оценки требуемой метрики ранжирования NDCG_5.

Из анализа датасета, удалось установить, что количество документов, соответсвующих поисковой сессии, не имет фиксированную длину. А также может быть меньше 5.

In [64]:
from sklearn.metrics import ndcg_score
from sklearn.metrics import accuracy_score


def NDCG_atK_score(test_Data, pred_Data, k):
  query_ids = test_Data['query_id'].unique()
  test_Data_ranks = np.array([[test_Data[test_Data['query_id'] == id]['rank'].iloc[i] for i in range(min(len(test_Data[test_Data['query_id'] == id]), k))] + [0 for _ in range(max(0, k - len(test_Data[test_Data['query_id'] == id])))] for id in query_ids])
  pred_Data_ranks = np.array([[pred_Data[pred_Data['query_id'] == id]['rank'].iloc[i] for i in range(min(len(pred_Data[pred_Data['query_id'] == id]), k))] + [0 for _ in range(max(0, k - len(pred_Data[pred_Data['query_id'] == id])))] for id in query_ids])
  return ndcg_score(test_Data_ranks, pred_Data_ranks)

### Попробуем применить модели классификации без какой-либо предобработки данных

Например, возьмем ансабль на деревьях RandomForestClassifier из библиотеки scikit-learn

In [ ]:
!pip install catboost

In [57]:
from sklearn.ensemble import RandomForestClassifier

In [58]:
RFC = RandomForestClassifier()
RFC.fit(df_train.drop(columns=['rank', 'query_id']), df_train['rank'])

RandomForestClassifier()

In [67]:
RFC_pred = RFC.predict(df_test.drop(columns=['rank', 'query_id']))
df_tmp = df_test.copy()
df_tmp['rank'] = RFC_pred
print(NDCG_atK_score(df_test, df_tmp, k=5))
print(NDCG_atK_score(df_test, df_tmp, k=15))


0.9043281179672613
0.8514035228749767


Так как NDCG@k принимает значения из диапазона $[0, 1]$, при этом лучше, чем ближе к 1, то можно сказать, что модель справилась хорошо. Однако, если не ограничиваться k=5, а посмотреть, например, на k=15, то заметим, что показатель падает. Кроме того относительно высокий показатель метрики, связан с дисбалансом рангов (анализ данных показал, что большая часть объектов имеет ранг 0). Посмотрим на показатель accuracy модели.

In [68]:
accuracy_score(df_test['rank'], df_tmp['rank'])

0.5453559435162589

Accuracy_score получился довольно низкий, модель зачастую ошибается в предсказании метки ранга. При этом преобладание идет к тому, что модель в основном выдает метку 0. И как было сказано ранее из-за дисбаланса класса вполне справляется с ранжированием в среднем, но часто может допускать значимые ошибки.

### Попробуем использовать градиентный бустинг

Теперь также используем данные о категориальных переменных, которые были получены при анализе датасета.

In [71]:
from catboost import CatBoostClassifier

to_drop = ['feature_14', 'feature_17', 'feature_19', 'feature_20',
       'feature_29', 'feature_34', 'feature_35', 'feature_44',
       'feature_74', 'feature_79', 'feature_84', 'feature_89',
       'feature_94', 'feature_114', 'feature_119', 'feature_120',
       'feature_121', 'feature_122', 'feature_123', 'feature_124',
       'feature_142', 'feature_143'] # Удалим данные, которые имеют сверхвысокую корреляцию между собой

df_train = df_train.drop(columns=to_drop)
df_test = df_test.drop(columns=to_drop)

cat_features = ['feature_95', 'feature_96', 'feature_97', 'feature_98', 'feature_99', 'feature_1', 'feature_3', 'feature_28', 'feature_33'] # категориальные признаки
for feature in cat_features:
  df_train[feature] = df_train[feature].astype(int)
  df_test[feature] = df_test[feature].astype(int)


CBC = CatBoostClassifier(verbose=False, cat_features=cat_features)

In [72]:
CBC.fit(df_train.drop(columns=['rank', 'query_id']), df_train['rank'], cat_features=cat_features)

In [74]:
y_pred = CBC.predict(df_test.drop(columns=['rank', 'query_id']))

df_tmp = df_test.copy()
df_tmp['rank'] = y_pred

print(NDCG_atK_score(df_test, df_tmp, k=5))
print(NDCG_atK_score(df_test, df_tmp, k=15))
print(accuracy_score(df_test['rank'], y_pred))

0.9080620009319861
0.8596031358002284
0.5580168116304779
